In [85]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/LEVEL 1/torch')

from multiclass_classification1 import *
import torch
from torch import nn, optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cpu


In [86]:
BATCH_SIZE = 32
LR = 1e-3
EPOCH = 5
criterion = nn.CrossEntropyLoss()
new_model_train = False
model_type ="MLP"
dataset = "MNIST"
save_model_path = f"/content/drive/MyDrive/Colab Notebooks/results/{model_type}_{dataset}.pt"

In [87]:
transform = transforms.ToTensor()
train_DS = datasets.MNIST(root='/content/drive/MyDrive/Colab Notebooks/data',train=True, download=True,transform=transform)
test_DS = datasets.MNIST(root='/content/drive/MyDrive/Colab Notebooks/data',train=False, download=True,transform=transform)
train_DL = torch.utils.data.DataLoader(train_DS, batch_size=BATCH_SIZE, shuffle=True)
test_DL = torch.utils.data.DataLoader(test_DS, batch_size=BATCH_SIZE, shuffle=True)

In [88]:
class MLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.fcs = nn.Sequential(
            nn.Linear(28*28, 100),
            nn.ReLU(),
            nn.Linear(100, 10)
        )
  def forward(self,x):
    x = torch.flatten(x, start_dim=1)
    x = self.fcs(x)
    return x

In [89]:
model = globals()[model_type]().to(DEVICE) #globals()[model_type]는 'MLP'라는 string으로 인덱싱 한것.
print(model)
x_batch, _ = next(iter(train_DL))
print(x_batch.shape)
print(model(x_batch.to(DEVICE)).shape)

MLP(
  (fcs): Sequential(
    (0): Linear(in_features=784, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=10, bias=True)
  )
)
torch.Size([32, 1, 28, 28])
torch.Size([32, 10])


In [90]:
if new_model_train:
  optimizer = optim.Adam(model.parameters(), lr = LR)
  loss_history = Train(model, train_DL, criterion, optimizer, EPOCH)

  torch.save(model.state_dict(), save_model_path)
  plt.plot(range(1, EPOCH+1), loss_history)
  plt.xlabel("Epochs")
  plt.ylabel("Loss")
  plt.title("Train Loss")
  plt.grid()

In [91]:
load_model = torch.load(save_model_path, map_location=DEVICE)

In [95]:
model = MLP().to(DEVICE)
model.load_state_dict(torch.load(save_model_path, map_location=DEVICE))
model.eval()

Test(model, test_DL)

Test accuracy: 9753/10000 (97.5 %)


97.5

In [ ]:
ㅅ